# GridSearchCV and RandomSearchCV
Selecting hypermeters is always a challenge task in machine learning. Gridsearch and Randomizedsearch are two well known methods to find the best parameters. 

In GridsearchCV (Gridsearch Cross Validation) we use all the possible combinations of the parameters. But in RandomizedCV (Randomizedch Cross Validation), as it's name suggest, we select the combinations randomly. 


It is worth mentioning that, GridsearchCV is appropriate where:

- The training size is small (1,000 to 10,000). Otherwise it will take too long time to find parameters
- When the paremeters are equally important for the model



RandomizedCV is usable where:
- Training size is too big and it requires a long time to train
- When one or more parameters are more important than others

In [1]:
# Now we will implement both of the methods on a simple ML problem and evaluate their performance
# First load a preloaded dataset of sklearn

from sklearn import svm, datasets
iris = datasets.load_iris()

import pandas as pd
df = pd.DataFrame(iris.data, columns = iris.feature_names)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [2]:
# In iris dataset the target variable is not binded with iris.data. We have to collect it separately
# Targets are in one hot encoded form. We need to bring out the names from that

df['target'] = iris.target
df['flower'] = df['target'].apply(lambda x : iris.target_names[x])
df['flower'].unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [3]:
# Let's split the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(labels=['flower', 'target'], axis = 1),
                                                    df['flower'], test_size = 0.2)

In [4]:
# Now we will create our ML model with a random combination of parameters

svc_model = svm.SVC(kernel='linear', C = 0.02, gamma='auto')
svc_model.fit(X_train, y_train)
svc_model.score(X_test, y_test)

0.9666666666666667

In [5]:
# As the dataset is very small, we can use cross validation to see if the model is overfitted or not
from sklearn.model_selection import cross_val_score

In [6]:
cross_val_score(svm.SVC(kernel='linear', C = 0.02, gamma='auto'), 
                 (df.drop(labels=['flower', 'target'], axis = 1)), df['flower'], cv = 5) 

array([0.9       , 0.96666667, 0.86666667, 0.93333333, 1.        ])

In [8]:
# The accuracy result is consistent in cross validation, so it is not overfitted
# Now we can proceed to do GridsearchCV

from sklearn.model_selection import GridSearchCV

In [10]:
grid_search = GridSearchCV(svm.SVC(gamma='auto'),
                        {
                            'C' : [0.01, 1, 10],
                            'kernel' : ['rbf', 'linear']
                        },
                        cv = 5,
                        return_train_score = False
                       )

grid_search.fit((df.drop(labels=['flower', 'target'], axis = 1)), df['flower'])
grid_result_df = pd.DataFrame(grid_search.cv_results_)
grid_result_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002898,0.000495,0.001994,0.000003,0.01,rbf,"{'C': 0.01, 'kernel': 'rbf'}",0.900000,0.933333,0.900000,0.933333,1.0,0.933333,0.036515,5
1,0.003388,0.001493,0.001801,0.000402,0.01,linear,"{'C': 0.01, 'kernel': 'linear'}",0.900000,0.966667,0.866667,0.966667,0.9,0.920000,0.040000,6
2,0.002199,0.000398,0.001792,0.000752,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.001795,0.000399,0.000996,0.000002,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
4,0.001603,0.000486,0.001194,0.000393,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
5,0.001595,0.000489,0.001597,0.000488,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.966667,1.0,0.973333,0.038873,4


In [12]:
print('Best accuracy found in GridsearchCV:', grid_search.best_score_)
print('Best parameter combinaiton found in GridsearchCV:',grid_search.best_params_)

Best accuracy found in GridsearchCV: 0.9800000000000001
Best parameter combinaiton found in GridsearchCV: {'C': 1, 'kernel': 'rbf'}


In [13]:
# Now let's try for a RandomizedCV approach

from sklearn.model_selection import RandomizedSearchCV

In [17]:
# Now I will create a model for randomizedCV
# The only difference is here we have to define n_iter which is the number of 
    # model combinaitons will look for

    
rand_search = RandomizedSearchCV(svm.SVC(gamma='auto'),
                              {
                                  'C' : [0.01, 1, 10],
                                  'kernel' : ['rbf', 'linear', 'poly']
                              },
                              cv = 5,
                              return_train_score = False,
                              n_iter = 5
                             )
rand_search.fit((df.drop(labels=['flower', 'target'], axis = 1)), df['flower'])
rand_result_df = pd.DataFrame(rand_search.cv_results_)
rand_result_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003393,0.001854,0.001594,4.869572e-04,linear,0.01,"{'kernel': 'linear', 'C': 0.01}",0.900000,0.966667,0.866667,0.966667,0.9,0.920000,0.040000,5
1,0.006582,0.003431,0.002394,4.877495e-04,poly,10,"{'kernel': 'poly', 'C': 10}",1.000000,1.000000,0.900000,0.933333,1.0,0.966667,0.042164,3
2,0.002395,0.001018,0.001200,3.971677e-04,rbf,0.01,"{'kernel': 'rbf', 'C': 0.01}",0.900000,0.933333,0.900000,0.933333,1.0,0.933333,0.036515,4
3,0.001803,0.000389,0.001195,3.996959e-04,linear,10,"{'kernel': 'linear', 'C': 10}",1.000000,1.000000,0.900000,0.966667,1.0,0.973333,0.038873,2
4,0.001994,0.000002,0.000997,5.223489e-07,rbf,10,"{'kernel': 'rbf', 'C': 10}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1


In [19]:
print('Best accuracy found in RandomizedsearchCV:', rand_search.best_score_)
print('Best parameter combinaiton found in RandomizedsearchCV:', rand_search.best_params_)

Best accuracy found in RandomizedsearchCV: 0.9800000000000001
Best parameter combinaiton found in RandomizedsearchCV: {'kernel': 'rbf', 'C': 10}


In [15]:
# Till now we have searched among different parameters of the same model
# Now we will search different parameter combinaitons among different models


from sklearn.ensemble import RandomForestClassifier
all_models = {
    'svm' : {
        'model' : svm.SVC(gamma='auto'),
        'parameters' : {
            'C' : [0.01, 1, 10],
            'kernel' : ['rbf', 'linear', 'poly']
        }
    },
    'random_forest' : {
        'model' : RandomForestClassifier(),
        'parameters' : {
            'n_estimators' : [1, 10, 50]
        }
    }
}

In [20]:
# Now we will find the best accuracies and combinations


final_accuracy = []
for model_name, model_desc in all_models.items():
    clf_model = RandomizedSearchCV(model_desc['model'], model_desc['parameters'], cv = 5,
                                  return_train_score = False, n_iter = 3)
    clf_model.fit((df.drop(labels=['flower', 'target'], axis = 1)), df['flower'])
    final_accuracy.append(
    {
        'model' : model_name,
        'best accuracy' : clf_model.best_score_,
        'best parameters' : clf_model.best_params_
    })
    
final_result_df = pd.DataFrame(final_accuracy)
final_result_df

,model,best accuracy,best parameters
0,svm,0.98,"{'kernel': 'rbf', 'C': 1}"
1,random_forest,0.96,{'n_estimators': 50}


### Applying hyperparameter searching techniques on NN
In the following step, Gridsearch and Randomsearch approaches will be applied on Neural Network for parameter tuning. 

In [78]:
# Importing the important libraries and modules for this project

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf


from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Model
from keras.wrappers.scikit_learn import KerasClassifier

In [25]:
# Loading the dataset

df = pd.read_csv('heart_failure_clinical_records_dataset.csv')
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [26]:
# Let's first separate the independant and dependant variables

X=df.drop(["DEATH_EVENT"],axis=1)
y=df["DEATH_EVENT"]

In [27]:
# Now scaling the independant variables

col_names = list(X.columns)
st_scaler = preprocessing.StandardScaler()
X_df = st_scaler.fit_transform(X)
X_st = pd.DataFrame(X_df, columns = col_names)
X_st.describe().T

,count,mean,std,min,25%,50%,75%,max
age,299.0,5.703353e-16,1.001676,-1.754448,-0.828124,-0.070223,0.771889,2.877170
anaemia,299.0,1.009969e-16,1.001676,-0.871105,-0.871105,-0.871105,1.147968,1.147968
creatinine_phosphokinase,299.0,0.000000e+00,1.001676,-0.576918,-0.480393,-0.342574,0.000166,7.514640
diabetes,299.0,9.060014e-17,1.001676,-0.847579,-0.847579,-0.847579,1.179830,1.179830
ejection_fraction,299.0,-3.267546e-17,1.001676,-2.038387,-0.684180,-0.007077,0.585389,3.547716
high_blood_pressure,299.0,0.000000e+00,1.001676,-0.735688,-0.735688,-0.735688,1.359272,1.359272
platelets,299.0,7.723291e-17,1.001676,-2.440155,-0.520870,-0.013908,0.411120,6.008180
serum_creatinine,299.0,1.425838e-16,1.001676,-0.865509,-0.478205,-0.284552,0.005926,7.752020
serum_sodium,299.0,-8.673849e-16,1.001676,-5.363206,-0.595996,0.085034,0.766064,2.582144
sex,299.0,-8.911489e-18,1.001676,-1.359272,-1.359272,0.735688,0.735688,0.735688


In [28]:
# Now the dataset values are prepared to be fed into a neural network
# Let's split the dataset into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X_st, y, test_size = 0.25, random_state = 101)
X_train.shape, y_train.shape, X_test.shape

((224, 12), (224,), (75, 12))

In [108]:
# Now creating a simple neural network


def create_model(neurons_l1=1, neurons_l2=1, dropout_rate=0.0, activation_func='relu'):


    input1 = Input(shape=(X_train.shape[1],))
    hidden1 = Dense(neurons_l1, activation=activation_func)(input1)
    hidden11 = Dropout(dropout_rate)(hidden1)
    hidden2 = Dense(neurons_l2, activation=activation_func)(hidden11)
    output = Dense(1, activation='sigmoid')(hidden2)
    model_func = Model(inputs=input1, outputs=output)
    model_func.compile(loss='binary_crossentropy', 
                   optimizer='adam', metrics=['accuracy']
                   )
    return model_func

In [109]:
# Creating Kerasclassifier for grid searching

model_func = KerasClassifier(build_fn=create_model, epochs=50, batch_size=15, verbose=0 )


# These are the hyperparameter values we want to check

neurons = [1, 5, 10, 15, 20, 25, 30]
dropout_rates = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
activations = ['softmax', 'relu', 'tanh', 'sigmoid']

# Building the gridsearch model and finding the best parameters
param_grid = dict(neurons_l1=neurons, neurons_l2=neurons, dropout_rate=dropout_rates, activation_func = activation) 
grid = GridSearchCV(estimator=model_func, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

In [121]:
# So the best result along with the parameter combinations are:


print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# zipped = list(zip(means, stds, params))
# print (max(zipped,key=lambda x:x[0]))


Best: 0.821141 using {'activation_func': 'relu', 'dropout_rate': 0.5, 'neurons_l1': 15, 'neurons_l2': 20}
